**Diffusion Pipeline**

Huggingface: https://huggingface.co/DamarJati

Github: https://github.com/Damarcreative

Instagram: @Damar_shop & @Pixonelab

Mail: dev@damarcreative.my.id

Site: www.damarcreative.my.id

------


In [ ]:
#@title Setup
!git clone https://github.com/Damarcreative/anime-tts.git
%cd anime-tts
!pip install -r requirements-googlecolab.txt
!mkdir model

import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write

def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

hps = utils.get_hparams_from_file("configs/config-single-speaker.json")

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()



In [ ]:
import os
import requests
from tqdm import tqdm
import utils  # Assuming you have a utility function for loading checkpoints

models = [
    "abyssinvoker.pth", "alice.pth", "ameth.pth", "asuna.pth", "ayaka-jp.pth", "azusa.pth", "bronya.pth",
    "chisato.pth", "doom.pth", "echo.pth", "eriko.pth", "eula.pth", "hatsune.pth", "herta.pth", "hina.pth",
    "hiyori.pth", "hoshino.pth", "iori.pth", "iroha.pth", "izuna.pth", "kafka.pth", "karin.pth", "keqing.pth",
    "kokoro.pth", "kyaru.pth", "kyoka.pth", "mika.pth", "misora.pth", "miyu.pth", "momoi.pth", "nahida-jp.pth",
    "pecorine.pth", "shiroko.pth", "takina.pth", "theresa.pth", "yuni.pth", "yuuka.pth", "zenyatta.pth"
]

model_name = "mika"   # @param [ "abyssinvoker", "alice", "ameth", "asuna", "ayaka-jp", "azusa", "bronya", "chisato", "doom", "echo", "eriko", "eula", "hatsune", "herta", "hina", "hiyori", "hoshino", "iori", "iroha", "izuna", "kafka", "karin", "keqing", "kokoro", "kyaru", "kyoka", "mika", "misora", "miyu", "momoi", "nahida-jp", "pecorine", "shiroko", "takina", "theresa", "yuni", "yuuka", "zenyatta" ]

model_path = f"./model/{model_name}.pth"

if not os.path.exists(model_path):
    model_url = f"https://huggingface.co/tensor-diffusion/anime-tts/resolve/main/anime-tts-model/{model_name}.pth"
    print(f"Downloading {model_name} model...")

    response = requests.get(model_url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 KB
    progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

    with open(model_path, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)

    progress_bar.close()

    if total_size != 0 and progress_bar.n != total_size:
        print("Error in downloading. Please try again.")
        os.remove(model_path)
    else:
        print("Download complete!")

# Assuming you have a utility function like utils.load_checkpoint
# Replace utils.load_checkpoint with your actual function
_ = utils.load_checkpoint(model_path, net_g, None)


100%|██████████| 160M/160M [00:02<00:00, 58.5MiB/s]


Download complete!


In [ ]:
#Generate Audio
text = "こんにちは。何かお手伝いできることはありますか?" # @param {type:"string"}

Noise_Scale = 667 # @param {type:"number"}
Noise_Scale_w = 0.8 # @param {type:"number"}
length_scale = 1.2 # @param {type:"number"}

Noise_scale = float(f"0.{Noise_Scale}")
stn_tst = get_text(text, hps)
print("Model Name:" + model_name)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=Noise_scale, noise_scale_w=Noise_Scale_w, length_scale=length_scale)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

Model Name:mika
